In [1]:
import tarfile
tf = tarfile.open("UTKFace.tar.gz", mode="r")
tf.extractall(members=None)

In [68]:
from PIL import Image
import numpy as np
import shutil
import matplotlib.pyplot as plt
import os
import torch.nn as nn
from torch.autograd import Variable
import torchvision
from torchvision import models,transforms,datasets
import torch
#import bcolz
import time
%matplotlib inline
import pandas as pd
import random
import torch.nn.functional as F

In [4]:
os.chdir("UTKFace")
os.mkdir('age')
os.mkdir('race')
os.mkdir('gender')

In [5]:
os.chdir('gender')
os.mkdir('train')
os.mkdir('valid')
os.chdir('train')
os.mkdir('male')
os.mkdir('female')
os.chdir("..")
os.chdir('valid')
os.mkdir('male')
os.mkdir('female')
os.chdir("..")
os.chdir("..")

In [6]:
os.chdir('age')
os.mkdir('train')
os.mkdir('valid')
os.chdir('train')
for i in range (1,107):
    os.mkdir('age'+str(i))
os.chdir("..")
os.chdir('valid')
for i in range (107):
    os.mkdir('age'+str(i))
os.chdir("..")
os.chdir("..")

In [7]:
os.chdir('race')
os.mkdir('train')
os.mkdir('valid')
os.chdir('train')
os.mkdir('white')
os.mkdir('black')
os.mkdir('asian')
os.mkdir('indian')
os.mkdir('others')
os.chdir("..")
os.chdir('valid')
os.mkdir('white')
os.mkdir('black')
os.mkdir('asian')
os.mkdir('indian')
os.mkdir('others')
os.chdir("..")
os.chdir("..")

In [8]:
def extract_labels(wordtest,rate) :
    suffix=".jpg.chip.jpg"
    wordtest=wordtest[:-len(suffix)]
    #(age,gender,racle,datetimz)=wordtest.split("_")
    lstr=wordtest.split("_")
    age=lstr[0]
    sex="male"
    if lstr[1]==str(1):
        sex="female"
    nb=lstr[2]
    race='white'
    if (nb==1):
        race='black'
    if (nb==2):
        race='asian'
    if (nb==3):
        race='indian'
    if (nb==4):
        race='others'        
    val="train"
    x=random.uniform(0, 1)
    if x<rate:
        val="valid"
    return(age,sex,race,val)

In [13]:
l=os.listdir()
l.sort()
l=l[:-3]

for i in range(len(l)):
    h=extract_labels(l[i],0.25)
    age,sex,race,val=h[0],h[1],h[2],h[3]
    desti1="gender/"+val+"/"+sex
    shutil.copy(src=l[i], dst=desti1)
    desti2="age/"+val+"/"+'age'+age
    shutil.copy(src=l[i], dst=desti2)
    desti3="race/"+val+"/"+race
    shutil.copy(src=l[i], dst=desti3)

    

In [49]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
im_format = transforms.Compose([transforms.CenterCrop(224), transforms.ToTensor(), normalize])


In [69]:
os.chdir('gender')
dsets = {x: datasets.ImageFolder(os.path.join(os.getcwd(), x), im_format) for x in ['train', 'valid']}
os.chdir('..')
dsets_loader = {x: torch.utils.data.DataLoader(dsets[x], batch_size=64, shuffle=True, num_workers=6) for x in ['train', 'valid']}

In [70]:
dset_sizes = {x: len(dsets[x]) for x in ['train', 'valid']}
print(dset_sizes)

{'train': 17734, 'valid': 5974}


In [64]:
net = nn.Sequential(nn.Linear(3 * 32 * 32, 1000), nn.ReLU(), nn.Linear(1000, 10))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr = 0.01, momentum=0.9, weight_decay=1e-4)

In [75]:
for data in dsets_loader['train']:
        x,y=data
        break

In [76]:
x = x[0]

In [80]:
x = np.array(x)
x = np.transpose(x)
x.shape

(3, 224, 224)

In [83]:
print(224*224*3)

150528


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()